In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = '../../bar_movement/data/'

In [3]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
df = pd.read_csv(file_path + 'Oanda_Eur_Usd_M5_2021-2022_test.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2021-09-22 06:00:00   1.17217   1.17218  1.17167    1.17168   1.17232   
1 2021-09-22 06:05:00   1.17169   1.17209  1.17163    1.17203   1.17182   
2 2021-09-22 06:10:00   1.17202   1.17226  1.17192    1.17222   1.17215   
3 2021-09-22 06:15:00   1.17225   1.17240  1.17216    1.17233   1.17237   
4 2021-09-22 06:20:00   1.17234   1.17246  1.17225    1.17228   1.17246   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  \
0   1.17232  1.17181    1.17181   1.17224   1.17224  1.17174    1.17174   
1   1.17223  1.17176    1.17216   1.17176   1.17216  1.17170    1.17210   
2   1.17241  1.17206    1.17236   1.17208   1.17234  1.17199    1.17229   
3   1.17256  1.17230    1.17245   1.17231   1.17248  1.17223    1.17239   
4   1.17259  1.17238    1.17242   1.17240   1.17252  1.17232    1.17235   

   Volume  
0     491  
1     267  
2     199  
3     320  
4     363  
--------------------------

In [5]:
def psar(barsdata, iaf=0.02, maxaf=0.2):
    length = len(barsdata)
    high = list(barsdata['Mid_High'])
    low = list(barsdata['Mid_Low'])
    close = list(barsdata['Mid_Close'])
    psar = close[0:len(close)]
    bull = True
    af = iaf
    hp = high[0]
    lp = low[0]
    for i in range(2, length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        reverse = False
        if bull:
            if low[i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = low[i]
                af = iaf
        else:
            if high[i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = high[i]
                af = iaf
        if not reverse:
            if bull:
                if high[i] > hp:
                    hp = high[i]
                    af = min(af + iaf, maxaf)
                if low[i - 1] < psar[i]:
                    psar[i] = low[i - 1]
                if low[i - 2] < psar[i]:
                    psar[i] = low[i - 2]
            else:
                if low[i] < lp:
                    lp = low[i]
                    af = min(af + iaf, maxaf)
                if high[i - 1] > psar[i]:
                    psar[i] = high[i - 1]
                if high[i - 2] > psar[i]:
                    psar[i] = high[i - 2]
    return psar


def atr(barsdata, lookback=14):
    high_low = barsdata['Mid_High'] - barsdata['Mid_Low']
    high_close = np.abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    low_close = np.abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).sum() / lookback


def rsi(barsdata, periods=14):
    close_delta = barsdata['Mid_Close'].diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))

    return rsi

  
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth


def stoch(high, low, close, lookback=14):
    high_lookback = high.rolling(lookback).max()
    low_lookback = low.rolling(lookback).min()
    slow_k = (close - low_lookback) * 100 / (high_lookback - low_lookback)
    slow_d = slow_k.rolling(3).mean()

    return slow_k, slow_d

def stoch_rsi(data, k_window=3, d_window=3, window=14):
    min_val = data.rolling(window=window, center=False).min()
    max_val = data.rolling(window=window, center=False).max()

    stoch = ((data - min_val) / (max_val - min_val)) * 100

    slow_k = stoch.rolling(window=k_window, center=False).mean()

    slow_d = slow_k.rolling(window=d_window, center=False).mean()

    return slow_k, slow_d

def n_macd(macd, macdsignal, lookback=50):
    n_macd = 2 * (((macd - macd.rolling(lookback).min()) / (macd.rolling(lookback).max() - macd.rolling(lookback).min()))) - 1
    n_macdsignal = 2 * (((macdsignal - macdsignal.rolling(lookback).min()) / (macdsignal.rolling(lookback).max() - macdsignal.rolling(lookback).min()))) - 1

    return n_macd, n_macdsignal

def chop(df, lookback=14):
    atr1 = atr(df, lookback=1)
    high, low = df['Mid_High'], df['Mid_Low']

    chop = np.log10(atr1.rolling(lookback).sum() / (high.rolling(lookback).max() - low.rolling(lookback).min())) / np.log10(lookback)

    return chop

def vo(volume, short_lookback=5, long_lookback=10):
    short_ema =  pd.Series.ewm(volume, span=short_lookback).mean()
    long_ema = pd.Series.ewm(volume, span=long_lookback).mean()

    volume_oscillator = (short_ema - long_ema) / long_ema

    return volume_oscillator

def bar_lengths(bar_lens, window=36):
    return bar_lens.rolling(window=window).mean(), bar_lens.rolling(window=window).std()

def sar_lengths(opens, sars, window=36):
    diffs = abs(opens - sars.shift(1))

    return diffs.rolling(window=window).mean(), diffs.rolling(window=window).std()

def range_high_lows(df, i, n_histograms=6):
    green, red = False, False
    curr_color, curr_streak = None, 0
    high, low = -np.inf, np.inf

    while i >= 0:
        hist = df.loc[df.index[i], 'macdhist']
        curr_high, curr_low = df.loc[df.index[i], ['Mid_High', 'Mid_Low']]
        new_color = 'green' if hist > 0 else 'red'

        if curr_color != new_color:
            curr_color = new_color
            curr_streak = 0

            if green and red:
                break

        curr_streak += 1

        if curr_streak >= n_histograms:
            high = max(high, curr_high)
            low = min(low, curr_low)

            if curr_color == 'red':
                red = True

            else:
                green = True

        i -= 1

    return high, low if (high > -np.inf and low < np.inf) else np.nan, np.nan

# def range_high_lows(df, i):
#     green, red = False, False
#     curr_color, curr_streak = None, 0
#     high, low = -np.inf, np.inf

#     while i >= 0:
#         hist = df.loc[df.index[i], 'macdhist']
#         curr_high, curr_low = df.loc[df.index[i], ['Mid_High', 'Mid_Low']]
#         new_color = 'green' if hist > 0 else 'red'

#         if curr_color != new_color:
#             curr_color = new_color
#             curr_streak = 0

#             if green and red:
#                 break

#         curr_streak += 1

#         if curr_streak >= 3:
#             if curr_color == 'red':
#                 low = min(low, curr_low)
#                 red = True

#             else:
#                 high = max(high, curr_high)
#                 green = True

#         i -= 1

#     return high, low if (high > -np.inf and low < np.inf) else np.nan, np.nan


In [6]:
# Add technical indicators (for additional features)
df['atr'] = atr(df)
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df)

                     Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0     2021-09-22 07:10:00   1.17315   1.17318  1.17293    1.17303   1.17327   
1     2021-09-22 07:15:00   1.17301   1.17317  1.17296    1.17313   1.17315   
2     2021-09-22 07:20:00   1.17312   1.17317  1.17302    1.17313   1.17326   
3     2021-09-22 07:25:00   1.17311   1.17318  1.17241    1.17287   1.17324   
4     2021-09-22 07:30:00   1.17286   1.17358  1.17286    1.17354   1.17299   
...                   ...       ...       ...      ...        ...       ...   
74905 2022-09-22 05:35:00   0.98376   0.98376  0.98345    0.98373   0.98392   
74906 2022-09-22 05:40:00   0.98377   0.98383  0.98323    0.98350   0.98391   
74907 2022-09-22 05:45:00   0.98349   0.98367  0.98343    0.98354   0.98362   
74908 2022-09-22 05:50:00   0.98352   0.98357  0.98327    0.98336   0.98369   
74909 2022-09-22 05:55:00   0.98338   0.98340  0.98235    0.98240   0.98354   

       Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_

In [8]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]
pair = 'EUR_USD'

In [9]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(open_price, stop_loss, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = abs(open_price - stop_loss)
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'JPY' else pips_to_risk * 100

    if second == 'USD':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'JPY' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, atr_multiplier, n_minutes, adx_cutoff):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    lo = None
    trade = None
    trade_lengths = []

    for i in range(1, len(df)):
        curr_date = df.loc[df.index[i], 'Date']
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, curr_mid_open, curr_mid_low = \
            df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                'Ask_Low', 'Ask_Close', 'Mid_Open', 'Mid_Low']]
        atr, adx, ask_low, bid_high = df.loc[df.index[i - 1], ['atr', 'adx', 'Ask_Low', 'Bid_High']]
        chop_signal = adx < adx_cutoff

        if lo is not None:
            minutes_elapsed = (curr_date - lo['start_date']).total_seconds() / 60
            stop_signal = adx >= adx_cutoff or minutes_elapsed >= n_minutes
            # stop_signal = minutes_elapsed >= n_minutes

            if stop_signal:
                lo = None

        if lo is None and trade is None and chop_signal:
            range_high, range_low = bid_high + (atr * atr_multiplier), ask_low - (atr * atr_multiplier)
            rang = abs(float(range_high) - float(range_low))
            pips_gain = (rang / 2) * risk_reward_ratio
            pips_to_risk = pips_gain / risk_reward_ratio
            
            if range_high > range_low and spread <= pips_to_risk * spread_cutoff:
                buy_open_price = float(range_low)
                buy_stop_gain = buy_open_price + (rang / 2) * risk_reward_ratio
                buy_pips_gained = buy_stop_gain - buy_open_price
                buy_pips_to_risk = buy_pips_gained / risk_reward_ratio
                buy_pullback = buy_open_price - buy_pips_to_risk
                buy_stop_loss = round(buy_pullback, 5)
                buy_mo = buy_open_price - (spread / 2)
                buy_n_units = get_n_units(buy_open_price, buy_stop_loss, buy_mo, pair)

                sell_open_price = float(range_high)
                sell_stop_gain = sell_open_price - (rang / 2) * risk_reward_ratio
                sell_pips_gained = sell_open_price - sell_stop_gain
                sell_pips_to_risk = sell_pips_gained / risk_reward_ratio
                sell_pullback = sell_open_price + sell_pips_to_risk
                sell_stop_loss = round(sell_pullback, 5)
                sell_mo = sell_open_price + (spread / 2)
                sell_n_units = get_n_units(sell_open_price, sell_stop_loss, sell_mo, pair)

                lo = {'buy_open_price': buy_open_price, 'buy_stop_gain': buy_stop_gain, 'buy_stop_loss': buy_stop_loss, 'buy_n_units': buy_n_units, 
                               'sell_open_price': sell_open_price, 'sell_stop_gain': sell_stop_gain, 'sell_stop_loss': sell_stop_loss, 'sell_n_units': sell_n_units, 'start_date': curr_date}

        if trade is None and lo is not None:
            place_buy = curr_ask_low <= lo['buy_open_price']
            place_sell = curr_bid_high >= lo['sell_open_price']

            if place_buy:
                open_price = lo['buy_open_price']
                stop_gain = lo['buy_stop_gain']
                pips_gained = stop_gain - open_price
                curr_pips_to_risk = pips_gained / risk_reward_ratio
                stop_loss = lo['buy_stop_loss']

                if stop_loss < open_price and stop_gain > open_price:
                    n_units = lo['buy_n_units']

                    trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                    'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                    'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None, 'index': i}

                    n_buys += 1

                    pips_risked.append(curr_pips_to_risk)
                    
                    lo = None

            elif place_sell:
                open_price = lo['sell_open_price']
                stop_gain = lo['sell_stop_gain']
                pips_gained = open_price - stop_gain
                curr_pips_to_risk = pips_gained / risk_reward_ratio
                stop_loss = lo['sell_stop_loss']

                if stop_loss > open_price and stop_gain < open_price:
                    n_units = lo['sell_n_units']

                    trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                            'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                            'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None, 'index': i}

                    n_sells += 1

                    pips_risked.append(curr_pips_to_risk)

                    lo = None

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1
            trade_lengths.append(i - trade['index'] + 1)

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1
            trade_lengths.append(i - trade['index'] + 1)

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1
            trade_lengths.append(i - trade['index'] + 1)

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1
            trade_lengths.append(i - trade['index'] + 1)

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, trade_lengths

In [11]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [0.25]
spread_cutoffs = [0.10, 0.20, 0.30]
atr_multipliers = [1.0, 1.5, 2.0]
n_minutes_vals = [5, 10, 15]
adx_cutoffs = [20, 25]

# risk_reward_ratio_vals = [0.5]
# spread_cutoffs = [0.2]
# atr_multipliers = [1.5]
# n_minutes_vals = [10]
# adx_cutoffs = [20]

all_combos = []


# use_half_sar, vo_cutoff, use_candles

for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs:
        for atr_multiplier in atr_multipliers:
            for n_minutes in n_minutes_vals:
                for adx_cutoff in adx_cutoffs:
                    all_combos.append((risk_reward_ratio, spread_val, atr_multiplier, n_minutes, adx_cutoff))

# percentage_to_try = 0.5
percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_atr_multiplier = None
best_n_minutes= None
best_adx_cutoff = None
top_n_results = 20
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, atr_multiplier, n_minutes, adx_cutoff in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, trade_lengths = run_simulation(risk_reward_ratio, spread_val, atr_multiplier, n_minutes, adx_cutoff)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_buys))
    print('Num sells: ' + str(n_sells))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    avg_pips_risked = (sum(pips_risked) / len(pips_risked)) if len(pips_risked) > 0 else 'N/A'
    avg_trade_len = (sum(trade_lengths) / len(trade_lengths)) if len(trade_lengths) > 0 else 0
    print('Avg pips risked: ' + str(avg_pips_risked))
    print('Avg trade length: ' + str(avg_trade_len))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread_val': spread_val, 'atr_multiplier': atr_multiplier, 'n_minutes': n_minutes, 'adx_cutoff': adx_cutoff})


    if reward > best_reward and avg_trade_len >= 1:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_atr_multiplier = atr_multiplier
        best_n_minutes = n_minutes
        best_adx_cutoff = adx_cutoff

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 54

1286.7068644644075
Num buys: 154
Num sells: 149
Num trades: 303
Num wins: 263
Num losses: 40
Win streak: 19
Loss streak: 2
Avg pips risked: 0.001108559641678476
Avg trade length: 1.0
Remaining runs: 53
Best reward so far: 1286.7068644644075

3917.6150817861408
Num buys: 561
Num sells: 518
Num trades: 1079
Num wins: 926
Num losses: 153
Win streak: 41
Loss streak: 3
Avg pips risked: 0.0009133142459949827
Avg trade length: 1.392029657089898
Remaining runs: 52
Best reward so far: 3917.6150817861408

1711.166223750192
Num buys: 297
Num sells: 290
Num trades: 587
Num wins: 497
Num losses: 90
Win streak: 22
Loss streak: 2
Avg pips risked: 0.001210285957653948
Avg trade length: 2.0664395229982966
Remaining runs: 51
Best reward so far: 3917.6150817861408

2562.3288830357615
Num buys: 190
Num sells: 165
Num trades: 355
Num wins: 325
Num losses: 30
Win streak: 44
Loss streak: 2
Avg pips risked: 0.0011925432595573501
Avg trade length: 1.0
Remaining runs: 50
Best reward so far: 3917.6

In [12]:
# EUR/USD
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best n minutes: ' + str(best_n_minutes))
print('Best adx cutoff: ' + str(best_adx_cutoff))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 11088.756349464844
Best risk/reward ratio: 0.25
Best spread: 0.3
Best atr multiplier: 1.0
Best n minutes: 5
Best adx cutoff: 25
-----------------------
Top results:
{'reward': 3917, 'ratio': 0.25, 'spread_val': 0.3, 'atr_multiplier': 1.5, 'n_minutes': 10, 'adx_cutoff': 20}
{'reward': 6249, 'ratio': 0.25, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_minutes': 5, 'adx_cutoff': 25}
{'reward': 5176, 'ratio': 0.25, 'spread_val': 0.3, 'atr_multiplier': 1.0, 'n_minutes': 10, 'adx_cutoff': 20}
{'reward': 11088, 'ratio': 0.25, 'spread_val': 0.3, 'atr_multiplier': 1.0, 'n_minutes': 5, 'adx_cutoff': 25}
{'reward': 7562, 'ratio': 0.25, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_minutes': 10, 'adx_cutoff': 25}
{'reward': 5038, 'ratio': 0.25, 'spread_val': 0.3, 'atr_multiplier': 1.5, 'n_minutes': 5, 'adx_cutoff': 25}
{'reward': 5371, 'ratio': 0.25, 'spread_val': 0.3, 'atr_multiplier': 1.0, 'n_minutes': 15, 'adx_cutoff': 25}
{'reward': 5304, 'rat

In [10]:
# GBP/USD
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best n minutes: ' + str(best_n_minutes))
print('Best adx cutoff: ' + str(best_adx_cutoff))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 4223.3566550000905
Best risk/reward ratio: 0.5
Best spread: 0.2
Best atr multiplier: 1.0
Best n bars: 2
-----------------------
Top results:
{'reward': 4124, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 1}
{'reward': 2249, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 2}
{'reward': 2668, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 3}
{'reward': 3674, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 1}
{'reward': 2172, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 2}
{'reward': 1450, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 3}
{'reward': 4223, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 2}
{'reward': 2473, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 3}
{'reward': 2275, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 1}


In [10]:
# AUD/USD
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best n bars: ' + str(best_n_bars))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 1876.0760714286596
Best risk/reward ratio: 0.5
Best spread: 0.2
Best atr multiplier: 1.0
Best n bars: 2
-----------------------
Top results:
{'reward': 95, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 2}
{'reward': 1798, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 1}
{'reward': 1876, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 2}
{'reward': 1650, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 3}
{'reward': 1700, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 1}
{'reward': 1497, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 2}
{'reward': 50, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 3}
{'reward': 850, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 3}
{'reward': 1199, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 1}


In [12]:
# USD/CAD
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best n bars: ' + str(best_n_bars))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 4048.6535253571433
Best risk/reward ratio: 0.5
Best spread: 0.2
Best atr multiplier: 1.0
Best n bars: 1
-----------------------
Top results:
{'reward': 2053, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 2}
{'reward': 716, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 2}
{'reward': 4048, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 1}
{'reward': 636, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 3}
{'reward': 523, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 3}
{'reward': 2977, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 3}
{'reward': 3480, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 2}
{'reward': 2039, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 1}
{'reward': 2445, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 1}


In [11]:
# USD/JPY
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best n bars: ' + str(best_n_bars))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 250758.18965857464
Best risk/reward ratio: 0.5
Best spread: 0.2
Best atr multiplier: 1.0
Best n bars: 1
-----------------------
Top results:
{'reward': 145619, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 3}
{'reward': 192316, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 2}
{'reward': 37162, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 3}
{'reward': 176101, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 2}
{'reward': 225827, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 1}
{'reward': 250758, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 1}
{'reward': 206984, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 2}
{'reward': 135342, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 1}
{'reward': 148785, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 3}


In [10]:
# USD/CHF
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best n bars: ' + str(best_n_bars))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 1456.2561146429143
Best risk/reward ratio: 0.5
Best spread: 0.2
Best atr multiplier: 1.5
Best n bars: 2
-----------------------
Top results:
{'reward': 1456, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 2}
{'reward': 1316, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 1}
{'reward': 225, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 3}
{'reward': 1034, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 1}
{'reward': 207, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 3}
{'reward': 690, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 1}
{'reward': 296, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 2}
{'reward': 302, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 2}
{'reward': 1339, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 3}


In [10]:
# NZD/USD
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best n bars: ' + str(best_n_bars))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 1074.2768450000149
Best risk/reward ratio: 0.5
Best spread: 0.2
Best atr multiplier: 1.0
Best n bars: 3
-----------------------
Top results:
{'reward': 725, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 1}
{'reward': 1074, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 3}
{'reward': -25, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 3}
{'reward': 624, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 2}
{'reward': 750, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 2}
{'reward': 449, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 1}
{'reward': 549, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 1}
{'reward': 774, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 3}
{'reward': 1049, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 2}


In [10]:
# GBP/CHF
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best n minutes: ' + str(best_n_minutes))
print('Best adx cutoff: ' + str(best_adx_cutoff))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 1054.763180803602
Best risk/reward ratio: 0.25
Best spread: 0.2
Best atr multiplier: 1.5
Best n minutes: 5
Best adx cutoff: 25
-----------------------
Top results:
{'reward': 206, 'ratio': 0.25, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_minutes': 10, 'adx_cutoff': 25}
{'reward': 178, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_minutes': 15, 'adx_cutoff': 25}
{'reward': 496, 'ratio': 0.25, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_minutes': 5, 'adx_cutoff': 20}
{'reward': 124, 'ratio': 0.5, 'spread_val': 0.1, 'atr_multiplier': 2.0, 'n_minutes': 10, 'adx_cutoff': 25}
{'reward': 124, 'ratio': 0.5, 'spread_val': 0.1, 'atr_multiplier': 2.0, 'n_minutes': 15, 'adx_cutoff': 20}
{'reward': 290, 'ratio': 0.25, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_minutes': 5, 'adx_cutoff': 25}
{'reward': 183, 'ratio': 0.25, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_minutes': 10, 'adx_cutoff': 15}
{'reward': 395, 'ratio': 0.5, 's

In [10]:
# EUR/JPY
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best n bars: ' + str(best_n_bars))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 403452.127566436
Best risk/reward ratio: 0.5
Best spread: 0.2
Best atr multiplier: 1.5
Best n bars: 2
-----------------------
Top results:
{'reward': 222544, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 1}
{'reward': 127405, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 2}
{'reward': 285747, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 1}
{'reward': 171298, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 2}
{'reward': 199570, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 3}
{'reward': 403452, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 2}
{'reward': 178521, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 1}
{'reward': 100253, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 3}
{'reward': 30516, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 3}
